# OPTIMIZATION 
## Diet Problem
### Find the optimized food combination meet the nutritional requirements while minimizing the cost.

In [1]:
#import PuLP and pandas modules

from pulp import *
import pandas as pd
import numpy as np

In [2]:
# load data
data = pd.read_excel("diet_large.xls", skiprows = 1, header = 0) # read all data, 1st row is blank

dataTable = data[0:7146] # rows 0:7146 (Excel calls them 2-7148; 
print(dataTable.tail(3))
dataTable = dataTable.values.tolist() # Convert dataframe to list
print(dataTable[:2])
nutrientNames = list(data.columns.values) # column headers (nutrient names are in columns 3-13; Excel calls them D-N)
print(nutrientNames)
numNutrients = len(nutrientNames) - 1 # don't count the food-name column
print(numNutrients)

                                              Long_Desc Protein  \
7143  Mollusks, scallop, (bay and sea), cooked, steamed    23.2   
7144                               Mollusks, snail, raw    16.1   
7145                                 Turtle, green, raw    19.8   

     Carbohydrate, by difference Energy Water  Energy.1 Calcium, Ca Iron, Fe  \
7143                           0    112  73.1     469.0         115        3   
7144                           2     90  79.2     377.0          10      3.5   
7145                           0     89  78.5     372.0         118      1.4   

     Magnesium, Mg Phosphorus, P  ... Riboflavin Niacin Pantothenic acid  \
7143            55           338  ...       0.06    1.3              NaN   
7144           250           272  ...       0.12    1.4              NaN   
7145            20           180  ...       0.15    1.1              NaN   

     Vitamin B-6 Folate, total Vitamin B-12 Vitamin K (phylloquinone)  \
7143         0.1            12  

In [3]:
# replace nan' with zero

for i in range(0,7146):
    for j in range(1,numNutrients):
        if np.isnan(dataTable[i][j]):
            dataTable[i][j] = 0

minVal = data[7147:7148].values.tolist() # minimum nutrient values
maxVal = data[7149:7151].values.tolist() # maximum nutrient values

In [4]:
print(minVal)
print(maxVal)

[[nan, 56, 130, 2400, 3700, 2400.0, 1000, 8, 270, 700, 4700, 1500, 11, 0.9, 2.3, 55, 900, 15, 200, 90, 0.0012, 1.3, 16, 5, 1.3, 400, 2.4, 120, nan, nan, nan]]
[[nan, 1000000, 1000000, 1000000, 1000000, 1000000.0, 2500, 45, 400, 4000, 1000000, 2300, 40, 10, 11, 400, 3000, 1000, 2000, 2000, 1000000, 1000000, 35, 1000000, 100, 1000, 1000000, 1000000, nan, nan, nan]]


In [5]:
# Extract individual vectors of data

foods = [j[0] for j in dataTable] #list of food names

cost = dict([(j[0], float(j[nutrientNames.index('Cholesterol')])) for j in dataTable]) # cost for each food

nutrients = []
for i in range(0,numNutrients): # for loop running through each nutrient
    nutrients.append(dict([(j[0], float(j[i+1])) for j in dataTable])) # amount of nutrient i in food j

In [6]:
# Create a linear program Problem 
# minimization (find the *lowest* cost), use "LpMinimize" 

prob = LpProblem('Food optimization', LpMinimize)

#  Define the variables, choose foodVarsfor each food. Lower limit of each variable is 0 (non negative)

foodVars = LpVariable.dicts("Foods", foods, 0)


# objective function 
prob += lpSum([cost[f] * foodVars[f] for f in foods]), 'Total Cost'

# Add constraints for each nutrient 
for i in range(0,numNutrients): 
    if (not np.isnan(minVal[0][i+1])) and (not np.isnan(maxVal[0][i+1])): # only write a constraint if upper and lower bounds exist
        print("adding constraint for " + nutrientNames[i+1])
        prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) >= minVal[0][i+1], 'min nutrient ' + nutrientNames[i+1]
        prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) <= maxVal[0][i+1], 'max nutrient ' + nutrientNames[i+1]
            

# Solve the optimization problem and Print the output
prob.solve()
print()
print("The optimizied food is")
for var in prob.variables():
    if var.varValue > 0:
        print(str(var.varValue)+" units of "+str(var).replace('Foods_','') )
print()
print("Total cholesterol = %f" % value(prob.objective))

adding constraint for Protein
adding constraint for Carbohydrate, by difference
adding constraint for Energy
adding constraint for Water
adding constraint for Energy.1
adding constraint for Calcium, Ca
adding constraint for Iron, Fe
adding constraint for Magnesium, Mg
adding constraint for Phosphorus, P
adding constraint for Potassium, K
adding constraint for Sodium, Na
adding constraint for Zinc, Zn
adding constraint for Copper, Cu
adding constraint for Manganese, Mn
adding constraint for Selenium, Se
adding constraint for Vitamin A, RAE
adding constraint for Vitamin E (alpha-tocopherol)
adding constraint for Vitamin D
adding constraint for Vitamin C, total ascorbic acid
adding constraint for Thiamin
adding constraint for Riboflavin
adding constraint for Niacin
adding constraint for Pantothenic acid
adding constraint for Vitamin B-6
adding constraint for Folate, total
adding constraint for Vitamin B-12
adding constraint for Vitamin K (phylloquinone)

The optimizied food is
0.059863415

# Alternative Solution

In [7]:
# maximize protein intake rather than minimize cholesterol 

cost = dict([(j[0], float(j[nutrientNames.index('Protein')])) for j in dataTable]) # cost for each food

# This problem is a maxmization problem , so "LpMaximize" 
prob = LpProblem('Food optimization', LpMaximize) # 2 parameters: "name" and "sense"


# Define the variables 
foodVars = LpVariable.dicts("Foods", foods, 0)


# Create objective function 
prob += lpSum([cost[f] * foodVars[f] for f in foods]), 'Total Cost'

#  Add constraints 

for i in range(0,numNutrients): 
    if (not np.isnan(minVal[0][i+1])) and (not np.isnan(maxVal[0][i+1])): # only write a constraint if upper and lower bounds exist
        print("adding constraint for " + nutrientNames[i+1])
        prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) >= minVal[0][i+1], 'min nutrient ' + nutrientNames[i+1]
        prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) <= maxVal[0][i+1], 'max nutrient ' + nutrientNames[i+1]
            

#Solve the optimization problem 

prob.solve()
print()
print("---------The solution to the diet problem is----------")
for var in prob.variables():
    if var.varValue > 0:
        print(str(var.varValue)+" units of "+str(var).replace('Foods_','') )
print()
print("Total protein = %f" % value(prob.objective))

adding constraint for Protein
adding constraint for Carbohydrate, by difference
adding constraint for Energy
adding constraint for Water
adding constraint for Energy.1
adding constraint for Calcium, Ca
adding constraint for Iron, Fe
adding constraint for Magnesium, Mg
adding constraint for Phosphorus, P
adding constraint for Potassium, K
adding constraint for Sodium, Na
adding constraint for Zinc, Zn
adding constraint for Copper, Cu
adding constraint for Manganese, Mn
adding constraint for Selenium, Se
adding constraint for Vitamin A, RAE
adding constraint for Vitamin E (alpha-tocopherol)
adding constraint for Vitamin D
adding constraint for Vitamin C, total ascorbic acid
adding constraint for Thiamin
adding constraint for Riboflavin
adding constraint for Niacin
adding constraint for Pantothenic acid
adding constraint for Vitamin B-6
adding constraint for Folate, total
adding constraint for Vitamin B-12
adding constraint for Vitamin K (phylloquinone)

---------The solution to the diet 